## Семинар 8: "Современные модели для NLP"

ФИО: __Смирнов Егор Геннадьевич__

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~150MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 11.2MB/s 


In [3]:
import torch
import numpy as np
!pip install --upgrade transformers
from torch.utils.data import RandomSampler
from torch.distributions.categorical import Categorical
from transformers import *

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.5.1)


In [4]:
MODEL = (MobileBertForMaskedLM, MobileBertTokenizer, 'google/mobilebert-uncased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 2182, 2003, 2070, 3793, 2000, 4372, 16044, 102]


In [ ]:
tokenizer.decode(input_ids)

'[CLS] here is some text to encode [SEP]'

In [ ]:
input_ids[-2] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] here is some text to en [MASK] [SEP]'

In [ ]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [ ]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [ ]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some text to enact.'

In [5]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

In [ ]:
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

Первый способ -- берем максимум вероятности

In [6]:
GPT_text = []

In [7]:
for text in GPT_TEXTS:
  res_text = text
  while len(res_text) < 1000:
      input_ids = tokenizer.encode(res_text, add_special_tokens=True)
      input_ids[-1] = tokenizer.mask_token_id
      input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
      with torch.no_grad():
          res = model(input_batch)[0]
      prob = torch.nn.functional.softmax(res, dim=-1)
      new_ids = prob.max(-1)[1]
      res_text = tokenizer.decode(new_ids.numpy()[0, :].tolist())[2:]
  GPT_text.append(res_text)

In [8]:
GPT_text[0]

'in a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. they also discovered a herd of wolves and coyotes living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountains. they also discovered a herd of sheep and goats living in a small valley in the andes mountai

In [9]:
GPT_text[1]

'a train carriage containing controlled nuclear materials was stolen in cincinnati today. its whereabouts are unknown. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carr

Берем рандомный семпл

In [10]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

In [29]:
GPT_text = []

In [30]:
for text in GPT_TEXTS:
  res_text = text
  while len(res_text) < 1000:
      input_ids = tokenizer.encode(res_text, add_special_tokens=True)
      input_ids[-1] = tokenizer.mask_token_id
      input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
      with torch.no_grad():
          res = model(input_batch)[0]
      prob = torch.nn.functional.softmax(res, dim=-1)
      m = Categorical(prob)
      predicted_index = m.sample()[0] 
      new_id = predicted_index[-1].item()
      res_text += ' ' + tokenizer.decode([new_id])
  GPT_text.append(res_text)

In [31]:
GPT_text[0]

'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. together , they combined our spanish and our english words and expressions to micro memory comprehend them . we saw colors distinctly different color . we saw different world and location people . different world and people . changed people and people . we knew the world and people . we knew people and people . we knew people and people . we saw differences between think and situations . we saw different environments and weather and weather and warmth and warmth and daylight and daylight and warm and warm . we saw different situations and situations and ve ##rmi # # - 000 ##3 sc ##z # # + # # # - # # number # # s # # # # # # # # # # # # # # # # # # # # # # # # # # # # - # # # # # # # 1 s = : student # 1 number 25 just alternate to notices # numbers 8 named a

In [32]:
GPT_text[1]

'A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. < ref > please visit nuclear sites at hr ##s ##x # jett # j # q # e # n # n # e # n # e # ne # n # ne # n traces # ne # n # e # n # ne # e # n # ne # n # e # also # n # e # n # e # ne # n # e # e # e # n # n # e # n # e # n # e # n # n # e # n # e # n # & e # n # n # e # n # for services include the following persons with sh , funded . . . : publisher who has created something significant alongside the record - as a result of the accident and the explosion - regarding the dead grave ; destroyed by gasoline , gasoline , and gasoline , gasoline and gasoline and gasoline and gasoline and gasoline and gasoline and gasoline . . . sa aa ##nch # sport # nc ##u # ny # nj # n # n # n # e teen n # n # n # e # n # n # e # n # e # n # e # n # n # n # n # e # n # ne # n # n # e # n # e # { n # ne # n # n slightly e # n # n # ne # n # n # e # n # n # ne # ne # n # n # n # e # n # ne

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: